In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ImageIngestionConfig:
    root_dir: Path
    url: str
    save_img_original: Path
    save_img_low_res: Path
    pca_components: int


    

In [3]:
from Image_enhance.constants import *
from Image_enhance.utils.common import read_yaml, create_directories
from Image_enhance import logger
import requests
import imageio
import cv2
from sklearn.decomposition import PCA
import numpy as np
import time

[2024-05-09 14:43:01,189: WARNING: module_wrapper: From c:\Users\Maza\Desktop\Image_enhance\venv\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
:]


In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        """
        Initializes ConfigurationManager with provided filepaths.

        Args:
            config_filepath (str): Filepath to configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (str): Filepath to parameters file. Defaults to PARAMS_FILE_PATH.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    
    def get_image_ingestion_config(self) -> ImageIngestionConfig:
        """
        Retrieves image ingestion configuration settings.

        Returns:
            image_ingestion_config (ImageIngestionConfig): Image ingestion configuration object.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        image_ingestion_config = ImageIngestionConfig(
            root_dir=config.root_dir,
            url=config.url,
            save_img_original=config.save_img_original,
            save_img_low_res=config.save_img_low_res,
            pca_components = self.params.PCA.PCA_COMPONENTS
        )

        return image_ingestion_config



In [5]:
class ImageProcessor:
    def __init__(self, config:ImageIngestionConfig):
        self.config = config

    def save_img_from_url(self):
        url = self.config.url
        self.save_original_img_path = self.config.save_img_original
        logger.info(f"Image obtained from : {url}")
        response = requests.get(url)
        with open(self.save_original_img_path, "wb") as f:
            f.write(response.content)
        logger.info(f"Image saved  in file: {self.save_original_img_path}")
        



    def decrese_img_resloution(self):
        logger.info(f"Decreasing the quality of the original image")
        save_low_res_img_path = self.config.save_img_low_res
        pca_components = self.config.pca_components

        img = cv2.cvtColor(cv2.imread(self.save_original_img_path),cv2.COLOR_BGR2RGB)
        logger.info(f"Image loaded from {self.save_original_img_path}")
        #split by componenets
        r,g,b = cv2.split(img)
        #normalize
        r,g,b = r/255, g/255, b/255

        #PCA components
        logger.info(f"Initializing PCA")
        pca_r = PCA(n_components=pca_components)
        reduced_r = pca_r.fit_transform(r)

        pca_g = PCA(n_components=pca_components)
        reduced_g = pca_g.fit_transform(g)


        pca_b = PCA(n_components=pca_components)
        reduced_b = pca_b.fit_transform(b)


        reconstructed_r = pca_r.inverse_transform(reduced_r)* 255
        reconstructed_g = pca_g.inverse_transform(reduced_g)* 255
        reconstructed_b = pca_b.inverse_transform(reduced_b)* 255
        img_reconstructed = (cv2.merge((reconstructed_r,reconstructed_g,reconstructed_b)))
        img_reconstructed_transfomred = img_reconstructed.astype(np.uint8)
        logger.info(f"Original image components: {img.shape}")
        logger.info(f"Low resolution image components: {img_reconstructed_transfomred.shape}")

        imageio.imwrite(save_low_res_img_path, img_reconstructed_transfomred)
        logger.info(f"Low resolution image loaded to {save_low_res_img_path}")




        



In [6]:
start = time.time()
image_ingestion_config = ConfigurationManager()
config = image_ingestion_config.get_image_ingestion_config()
image_processor = ImageProcessor(config)
image_processor.save_img_from_url()
image_processor.decrese_img_resloution()
logger.info(f"Image processor latency: {(time.time() - start):.4f} seconds")

[2024-05-09 14:43:03,237: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-05-09 14:43:03,239: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-05-09 14:43:03,242: INFO: common: created directory at: artifacts/data_ingestion:]
[2024-05-09 14:43:03,243: INFO: 1031633362: Image obtained from : https://user-images.githubusercontent.com/12981474/40157448-eff91f06-5953-11e8-9a37-f6b5693fa03f.png:]
[2024-05-09 14:43:03,602: INFO: 1031633362: Image saved  in file: artifacts/data_ingestion/monkey.png:]
[2024-05-09 14:43:03,602: INFO: 1031633362: Decreasing the quality of the original image:]
[2024-05-09 14:43:03,619: INFO: 1031633362: Image loaded from artifacts/data_ingestion/monkey.png:]
[2024-05-09 14:43:03,619: INFO: 1031633362: Initializing PCA:]
[2024-05-09 14:43:03,655: INFO: 1031633362: Original image components: (120, 125, 3):]
[2024-05-09 14:43:03,668: INFO: 1031633362: Low resolution image components: (120, 125, 3):]
[2024-05-09 14:43:03,750: I